In [2]:
import requests
import pyinputplus as pyip
from pprint import pprint
from requests import Response
from pydantic import BaseModel,RootModel,Field,ValidationError,field_validator
from random import choices

class UB(BaseModel):
	sna:str = Field(alias='sna') #  站點名稱
	sarea:str = Field(alias='sarea')# 站點區域
	mday:str = Field(alias='mday')# 資料更新時間
	ar:str  = Field(alias='ar') # 站點地址
	act:bool  # 是否可用
	total:int = Field(alias='total')# 總車位數
	available_rent_bikes:int = Field(alias='available_rent_bikes')# 可租借車輛數
	latitude:float  = Field(alias='latitude') # 緯度
	longitude:float  = Field(alias='longitude')# 經度
	available_return_bikes:int = Field(alias='available_return_bikes') # 可還車位數

class Root(RootModel):
    root:list[UB]

def main():
    youbike_url = 'https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json'
    res:Response = requests.get(youbike_url)
    all_data = res.json()

    try:
        root:Root = Root.model_validate(all_data)
    except ValidationError as error:
            print(error)

    data_list:list[dict] = root.model_dump()
    filter_list:list[dict] = list(filter(lambda item:False if item['act']==0 else True,data_list))
    filter_list_rent_less3:list[dict] = list(filter(lambda item:True if item['場站目前車輛數量']<3 else False,data_list))
    filter_list_return_less3:list[dict] = list(filter(lambda item:True if item['空位數量']<3 else False,data_list))

    print(f"台北市YouBike有{len(data_list)}站點，目前正常運作站點共{len(filter_list)}個")

    num = pyip.inputInt("請輸入隨機顯示的站點數(1~5): ", min=1, max=5)
    print(num)

    print(f"\n可借車輛數為3輛以內的站點有{len(filter_list_rent_less3)}站")
    num_list:list[dict] = choices(filter_list_rent_less3, k=num)
    for sta in num_list:
            seq = filter_list_rent_less3.index(sta)
            print(f"No.{seq}")
            pprint(sta, indent=2, sort_dicts=False)
    print(f"\n可還車輛數為3輛以內的站點有{len(filter_list_return_less3)}站")
    num_list:list[dict] = choices(filter_list_return_less3, k=num)
    for sta in num_list:
            seq = filter_list_return_less3.index(sta)
            print(f"No.{seq}")
            pprint(sta, indent=2, sort_dicts=False)

if __name__ == '__main__':
     main()

    

KeyError: '場站目前車輛數量'